# 추천 시스템
사용자(User)에게 관련된 아이템(item)을 추천해 주는 것.
* 간단하게 추천 로직을 생각해보자!

영화 추천을 예로 들어 보겠습니다. 한국 드라마, 영화와 로맨스를 좋아하는 A와 미국 드라마와 액션물을 좋아하는 B가 있다고 가정해 봅시다.
A: 한국 드라마/영화, 로맨스물
B: 미국 드라마/영화, 액션물
그럼 부부의 세계, 스파이더맨 파 프롬 홈, 타짜가 상영중일 때, A씨와 B씨에게 각각 어떤 걸 추천해 주면 좋아할까요? (단, 아래의 비디오 출시 시기는 모두 3개월 이내, 인기도 역시 동일하다고 가정하겠습니다.)

A씨에게는 부부의 세계를 B씨에게는 스파이더맨 파 프롬 홈을 추천해 주면 좋아할 것 같네요.

지금이야 이렇게 관련 상품이 적어서 쉽게 판별이 가능하지만, 우리가 이용하는 넷플릭스나 왓챠가 하나하나 눈으로 판별하며 추천해 주진 않겠죠?

실제 추천 시스템에서는 영화들을 이렇게 좌표 평면에 표현합니다. 물론 수식을 이용해 정교하게 그리지만 일단 대략 비슷한 것들끼리 한번 배치해 볼게요.

<img src="./assets/Recommendation.png" width=80%></img>

이렇게 놓고 봤을때, 거리가 좁으면 좁을수록 유사도가 높다고 생각할 수 있습니다. 즉, 사용자와 관련된 항목을 찾아 추천해 줄 수 있습니다.

영화만 이렇게 배치할까요? 자, 신규 사용자 C가 새로 가입했습니다. 보통의 추천 앱은 사용자가 가입할 때 취향을 체크하게끔 되어 있는데 C는 귀찮았는지 그냥 다 스킵해 버렸어요. 그래서 C에 대해선 개인 정보인 (나이, 성별, 국적, 직업, 사는곳) 밖에 없습니다.

* C: 21살/여성/대한민국/학생/서울

그리고 앞서 예로 들었던 A와 B의 개인 정보입니다.

* A: 32살/여성/대한민국/마케팅/인천
* B: 41살/남성/미국/군인/용산

C에게는 어떤 영화를 추천해 주면 좋을까요? 이럴 때는 다른 비슷한 조건의 사용자가 좋아했던 항목들을 바탕으로 추천을 해줄수 있습니다. C와 비슷한 사람은..A이니 A가 좋아했던 것 중에 추천을 해 줍니다. 혹은 A에게 추천한 부부의 세계를 추천해 주어도 좋을것 같네요.



여기서 중요한 사실!
1. 범주형(Categorical) 데이터를 다룬다.
* 액션물, 로맨스물, 스릴러물, 한국드라마, 미국드라마, 영국드라마 등의 영화 `item` 데이터와 A,B,C 같은 `user` 데이터는 연속적이지 않고 이산적(discrete)이다. 이를 범주형 데이터라고 한다.
2. (숫자 벡터로 변환한 뒤) 유사도를 계산한다.
* 범주형 데이터들을 좌표에 나타내었는데, 좌표에 나타내기 위해서는 숫자로 이루어진 벡터(numerical vector)로 변환해야 한다. 그리고 그 거리를 계산하여 유사도를 나타낸다.

## 코사인 유사도
범주형 데이터를 어떻게 벡터로 변환한 뒤 유사도를 계산하는가!
* 코사인 유사도(Cosine Similarity) : 두 벡터 간의 코사인 값을 이용해 두 벡터의 유사도를 계산. 두 벡터의 방향이 이루는 각에 코사인을 취해 구한다.
<img src="./assets/recom_1.png" width=80%></img>

numpy를 이용한 코사인 유사도 계산

In [1]:
import numpy as np

t1 = np.array([1, 1, 1])
t2 = np.array([2, 0, 1])

In [3]:
# 코사인 유사도 계산하기
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [4]:
cos_sim(t1, t2)

0.7745966692414834

sklearn의 코사인 유사도 모듈 임포트

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# 해당 모듈은 입력값으로 2차원 배열을 받기 때문에 2차원 배열로 정의한다.
t1 = np.array([[1, 1, 1]])
t2 = np.array([[2, 0, 1]])
cosine_similarity(t1,t2)

array([[0.77459667]])

## 추천 시스템의 종류
* 콘텐츠 기반 필터링(Content Based Filtering)
* 협업 필터링(Collaborative Filtering)
* 딥러닝 적용 방식 or 하이브리드 방식

### 콘텐츠 기반 필터링(Content Based Filtering)
어떤 사람이 한 영화를 좋아했다면, 그와 비슷한 콘텐츠의 아이템을 추천하는 방식. 순수하게 콘텐츠의 내용만을 비교해서 추천한다.
* 어떤 사람이 아이언맨1을 봤으면, 아이언맨2, 아이언맨3, 마블 영화를 추천해 주는 방식.
* 영화에서 비슷한 콘텐츠란?
    * 우리가 어떤 기준으로 영화를 고르는가!
    * 장르, 배우, 감독 등의 정보
    * 이러한 정보들이 영화의 **특성(feature)** 이 되고, 이 특성이 콘텐츠가 비슷하다고 말할 수 있는 요인이 된다.
* [콘텐츠 기반 필터링 실습에 참고한 자료](http://www.codeheroku.com/post.html?name=Building%20a%20Movie%20Recommendation%20Engine%20in%20Python%20using%20Scikit-Learn)

In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

* load data
```
wget https://aiffelstaticprd.blob.core.windows.net/media/documents/movie_dataset.csv
```

In [8]:
import os
csv_path = os.getenv('HOME')+'/workspace/Modulabs-Dasol/Study/assets/recommendation_movie/movie_dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [9]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [10]:
# 여기서는 아래 특성만 고려하여 유사도를 계산해보도록 하겠음!
features = ['keywords','cast','genres','director']
features

['keywords', 'cast', 'genres', 'director']

In [11]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

combine_features(df[:5])

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
dtype: object

In [12]:
for feature in features:
    df[feature] = df[feature].fillna('')

df["combined_features"] = df.apply(combine_features,axis=1)
df["combined_features"]

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

* Vectorization

이 칼럼을 벡터화한 후 코사인 유사도를 계산할 것. 특성컬럼으로 장르, 배우명, 감독명의 텍스트 데이터를 범주형 데이터로 보기 때문에 등장횟수를 세어 숫자 벡터로 만들 것이다. 사이킷런의 `CountVectorizer()`를 사용.

In [13]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print(type(count_matrix))
print(count_matrix.shape)
print(count_matrix)

<class 'scipy.sparse.csr.csr_matrix'>
(4803, 14845)
  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


* **CSR(Compressed Sparse Row) Matrix**

Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 메모리 사용량 최소화 & Sparse한 matrix와 동일한 행렬을 표현하는 데이터 구조!!!!!

* (0, 3115) 1라고 되어 있는 것은 1번째 row는 3116번째 단어가 1번 출현한다는 뜻.
* 이 데이터셋에는 총 14845개의 단어가 존재하는데, 이 단어들을 범주형으로 보고 그 단어의 출현 빈도만을 표시한 Matrix가 매우 Sparse하기 때문에 공간을 절약할 수 있는 형태로 표현한 것.

In [14]:
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)
print(cosine_sim.shape)

[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]
(4803, 4803)


In [15]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 3편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==3:
        break

Avatar와 비슷한 영화 3편은 

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1


### 협업 필터링(Collaborative Filtering)
과거의 사용자 행동 양식(User Behavior) 데이터를 기반으로 추천하는 방식.
* 사용자 기반
* 아이템 기반
* 잠재요인(latent factor) 방식

|user_id|item_id|rating|timestamp|
|:--:|:--:|:--:|:--:|
|196|242|3|881250949|
|186|302|3|891717742|
|22|377|1|878887116|
|244|51|2|880606923|
|166|346|1|8863397596|

위 데이터를 사용자와 아이템 간 interaction matrix로 변환

<img src="./assets/recom_2.png" width=80%></img>

이를 평점행렬이라고 부르기도 한다. 이처럼 행렬을 실제 데이터로 만들면 굉장히 희소(sparse)한 행렬이 만들어진다. 유튜브나 넷플릭스에 있는 몇 억 개의 동영상을, 몇 억 명의 사용자가 전부 봤을리 없기 때문! 따라서 대부분 평점에 대한 데이터는 0이고, 희소행렬이라 부른다.

### 협업 필터링 - 사용자 기반
```당신과 비슷한 고객들이 다음 상품을 구매했습니다```

### 협업 필터링 - 아이템 기반
```이 상품을 선택한 다른 고객들은 다음 상품을 구매했습니다```
* 아이템 간 유사도를 측정하여 해당 아이템을 추천하는 방식.
* 일반적으로 사용자 기반보다는 아이템 기반 방식이 정확도가 더 높다.

### 협업 필터링 - 잠재요인 기반
잠재요인 협업 필터링은 평점행렬을 행렬 인수분해(matrix factorization)을 통해 잠재요인(latent factor)을 분석.
* SVD(Singular Vector Decomposition)
* ALS(Alternating Least Squares)
* NMF(Non-Negative Factorization)

#### SVD(Singular Vector DEcomposition) : 특잇값 분해
<img src="./assets/recom_3.png" width=90%></img>
* [SVD - 공돌이의 수학정리노트](https://angeloyeo.github.io/2019/08/01/SVD.html)
* [SVD의 활용 - 다크프로그래머](https://darkpgmr.tistory.com/106)
* SVD : 정보 복원을 위해 사용된다.
    *  특이값의 크기에 따라 A의 정보량이 결정되기 때문에 값이 큰 몇 개의 특이값들을 가지고도 충분히 유용한 정보를 유지할 수 있다. (중략) 최대한 중요한 정보들만 부분 복원해서 사용하면 사진의 용량은 줄어들지만 여전히 사진이 보여주고자 하는 내용은 살릴 수 있을 것이다.
* SVD Numpy 실습
    * [스펙확인](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)
    * svd의 리턴값은 (array) u, (array) s, (array) vh이다.

In [16]:
import numpy as np
from numpy.linalg import svd

In [17]:
np.random.seed(30)
A = np.random.randint(0, 100, size=(4, 4))
A

array([[37, 37, 45, 45],
       [12, 23,  2, 53],
       [17, 46,  3, 41],
       [ 7, 65, 49, 45]])

In [18]:
svd(A)

(array([[-0.54937068, -0.2803037 , -0.76767503, -0.1740596 ],
        [-0.3581157 ,  0.69569442, -0.13554741,  0.60777407],
        [-0.41727183,  0.47142296,  0.28991733, -0.72082768],
        [-0.6291496 , -0.46389601,  0.55520257,  0.28411509]]),
 array([142.88131188,  39.87683209,  28.97701433,  14.97002405]),
 array([[-0.25280963, -0.62046326, -0.4025583 , -0.6237463 ],
        [ 0.06881225, -0.07117038, -0.8159854 ,  0.56953268],
        [-0.73215039,  0.61782756, -0.23266002, -0.16767299],
        [-0.62873522, -0.47775436,  0.34348792,  0.50838848]]))

* 각각 행렬 U와 행렬 시그마, 행렬 V의 전치행렬이다.
* 이 값들을 unpacking해서 각각 변수명에 할당해 줄 것임.

In [19]:
U, Sigma, VT = svd(A)

print('U matrix: {}\n'.format(U.shape),U)
print('Sigma: {}\n'.format(Sigma.shape),Sigma)
print('V Transpose matrix: {}\n'.format(VT.shape),VT)

U matrix: (4, 4)
 [[-0.54937068 -0.2803037  -0.76767503 -0.1740596 ]
 [-0.3581157   0.69569442 -0.13554741  0.60777407]
 [-0.41727183  0.47142296  0.28991733 -0.72082768]
 [-0.6291496  -0.46389601  0.55520257  0.28411509]]
Sigma: (4,)
 [142.88131188  39.87683209  28.97701433  14.97002405]
V Transpose matrix: (4, 4)
 [[-0.25280963 -0.62046326 -0.4025583  -0.6237463 ]
 [ 0.06881225 -0.07117038 -0.8159854   0.56953268]
 [-0.73215039  0.61782756 -0.23266002 -0.16767299]
 [-0.62873522 -0.47775436  0.34348792  0.50838848]]


* 잘 분해가 되었따!
* 이제 다시 복원해 볼 것임.
* 복원을 위해서는 U, 시그마, V.T를 내적
* 시그마는 1차원이므로 0을 포함한 대각 행렬로 변환한 뒤 내적해야 한다.

In [20]:
Sigma_mat = np.diag(Sigma)

A_ = np.dot(np.dot(U, Sigma_mat), VT)
A_

array([[37., 37., 45., 45.],
       [12., 23.,  2., 53.],
       [17., 46.,  3., 41.],
       [ 7., 65., 49., 45.]])

#### Truncated SVD
추천 시스템에서의 행렬 인수분해는 SVD 중에서도 Truncated SVD를 사용.
* Truncated SVD : 잘린 SVD, 다른 말로 LSA(Latent Semantic Analysis : 잠재 의미 분석)
* Truncated SVD를 이용해 분해한 뒤 복원하면 SVD처럼 완벽히 같은 행렬이 나오지 않는다. Truncated SVD는 차원을 축소한 다음 행렬을 분해하기 때문.
    * [사이킷런 TruncatedSVD API reference](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)

SVD를 평가행렬에 적용하여 잠재요인을 분석하는 것은 아래와 같다.
<img src="./assets/recom_4.png" width=90%></img>

사용자가 아이템에 대한 평점을 매기는 요인으로는 많은 항목들이 있을 것이다. "배우가 마음에 들어서, 감독이 좋아서, 좋아하는 분위기라서..." 와 같이 평점을 매기는 것은 지극히 주관적이다.

따라서 사용자가 평점을 매기는 요인을 그냥 *잠재요인* 으로 취급한 뒤 그걸 SVD 기법을 이용해 분해한 뒤 다시 합치는 방법으로 영화에 평점을 매긴 이유를 벡터화하여 이를 기반으로 추천!

[행렬 인수분해와 추천 시스템 영상](https://youtu.be/ZspR5PZemcs)

이렇듯 협업 필터링을 이용하면 사용자가 아이템에 대해 평점을 매긴 평점행렬을 행렬 인수분해(Matrix Factorization)를 통해 잠재요인을 분석한 뒤 유사도를 계산할 수도 있고 사용자의 평점도 예측할 수 있다!!

## 실제 추천 시스템
사용자의 구매 여부와 평점 데이터 뿐만 아니라 얼마나 오랜 시간 동안 시청(혹은 해당 웹 사이트에 머물렀는지), 어떤 사이트에서 유입이 되었는지, 그리고 시청한 뒤 구매로 이어지기까지의 시간 등 우리의 족적들을 다 분석합니다. 이를 전문 용어로 Digital Footprint(디지털 발자국), Digital Shadow(디지털 그림자)라고 해요.

그리고 이중에서 가장 중요한 지표가 바로 **클릭률**입니다. 전문 용어로는 CTR(Click Through Rate)입니다. CTR은 마케팅에서도 중요한 지표로 작용하는 용어이기도합니다.

이러한 데이터들을 모아 추천을 한 뒤, 해당 아이템이 적절한 추천인지 여부를 평가하는 것 역시 중요한 일입니다. 추천한 제품이 구매로 이어졌는지를 통해 추천에 성공했는지를 평가하기도 하고 모델 단계에서 평가하기도 합니다.

이처럼, 추천 시스템은 굉장히 큰 시스템입니다. 데이터를 기반으로 사용자에게 적절한 제품을 추천한다는 것 그리고 그것이 구매로 이어지는것은 매출과 직결되는 문제이기도 합니다. 좋은 추천 시스템을 만들기 위해서는 어떤 데이터를 쓸지 많은 고민이 필요합니다. 사용자와 연관성이 있고, 구매와 직결되는 각종 데이터를 수집하고 정렬(sorting)하여 다시 순위(ranking)를 매긴 다음 평가하는 작업을 반복해가며 적합한 데이터와 추천 시스템을 만들어냅니다.

```
추천 시스템에 머신러닝이 적용될 수도 있는 것이지 머신러닝 안에 추천 시스템이 있는 것은 결코 아닙니다.
```

* [넷플릭스 추천시스템 - 유튜브](https://www.youtube.com/watch?v=f8OK1HBEgn0&feature=youtu.be)
* [왓챠 추천시스템 - 유튜브](https://youtu.be/jT-LJidbG5U)